# Download Searchanalytics Data
Herunterladen von Google Search Console Suchanalyse-Daten mit dem Python Package `searchconsole` [github link](https://github.com/joshcarty/google-searchconsole)

In [1]:
import searchconsole

Verifizieren des API Zugriffs.

→ Wenn ihr das Notebook auf Binder.org ausführt, werden eure Credentials natürlich nicht gespeichert!

In [2]:
account = searchconsole.authenticate(client_config='client_secrets.json')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=930130695988-uqe6l59cmd9qu96c60ti3u4cqq4vs0cr.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly&state=3HXuobCiS2NvHlyKWgqWdMDHJpiQ2t&access_type=offline


Liste der verifizierten Properties in dem Account

In [3]:
account.webproperties

[<searchconsole.account.WebProperty(url='https://www.serienjunkies.de/person/')>,
 <searchconsole.account.WebProperty(url='https://www.serienjunkies.de/news/')>,
 <searchconsole.account.WebProperty(url='https://www.serienjunkies.de/film/')>,
 <searchconsole.account.WebProperty(url='https://www.serienjunkies.de/')>]

Webproperty Object mit dem wir den Query erstellen.

In [4]:
webproperty = account['https://www.serienjunkies.de/']

Query erstellen und ausführen.

In [5]:
query = webproperty.query.range(start='2019-03-18', stop='2019-03-24') \
    .dimension('query')

In [6]:
data = query.get()

So sieht das ganze als Pandas DataFrame aus. Hiermit kann man nun alles mögliche anstellen.

In [7]:
df = data.to_dataframe()
df.head()

,query,clicks,impressions,ctr,position
0,serienjunkies,26323.0,32116.0,0.819623,1.010773
1,serienstream,4554.0,23731.0,0.191901,1.446757
2,star trek discovery,3112.0,34339.0,0.090626,2.645738
3,american gods,2426.0,55223.0,0.043931,4.252051
4,the walking dead staffel 9,2339.0,13989.0,0.167203,3.022232


Eine Abfrage mit Filter.

In [8]:
df = webproperty.query.range(start='2019-03-18', stop='2019-03-24') \
    .dimension('page','query') \
    .filter('page', 'https://www.serienjunkies.de/', 'equals') \
    .get() \
    .to_dataframe()
df.head()

,page,query,clicks,impressions,ctr,position
0,https://www.serienjunkies.de/,serienjunkies,23523.0,31990.0,0.735324,1.316255
1,https://www.serienjunkies.de/,serienjunkies.de,567.0,703.0,0.806543,1.159317
2,https://www.serienjunkies.de/,serienjunkie,382.0,577.0,0.662045,1.010399
3,https://www.serienjunkies.de/,serien junkies,104.0,217.0,0.479263,1.354839
4,https://www.serienjunkies.de/,mysj,74.0,86.0,0.860465,1.000000


Export der Daten als CSV. Die Datei sollte heruntergeladen werden können. ✌️😉

In [9]:
df.to_csv('gsc_searchanalytics_output.csv', header=True, index=False)